<a href="https://colab.research.google.com/github/mourodrigo/dataScienceCapstone/blob/master/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in New York City</font></h1>

## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in New York City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [241]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [242]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [0]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [0]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [0]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [246]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [0]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [248]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [249]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [250]:
city_name = 'Florianopolis, SC'

geolocator = Nominatim(user_agent="foursquare_explorer")
location = geolocator.geocode(city_name)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Florianopolis are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Florianopolis are -27.5973002, -48.5496098.


#### Create a map of New York with neighborhoods superimposed on top.

In [251]:
# create map of Manhattan using latitude and longitude values
city_map = folium.Map(location=[latitude, longitude], zoom_start=11)

city_map

Preparing the dataframe structure

Let's get the geographical coordinates of Florianopolis Neighborhoods.

In [252]:
!pip install wikipedia

import wikipedia
import requests
from bs4 import BeautifulSoup as bs
import time
import numpy as np
import pandas as pd

In [253]:
wiki = requests.get('https://pt.wikipedia.org/wiki/Lista_de_distritos_e_bairros_de_Florian%C3%B3polis')
soup = bs(wiki.text, 'lxml') 
table = soup.findAll("table",class_="wikitable")[1]
neighborhood_names = []
for items in table.find_all("tr")[:-1]:
    data = [' '.join(item.text.split()) for item in items.find_all(['th','td'])]
    neighborhood_names.append(data[1])

neighborhood_names.remove(neighborhood_names[0])
neighborhood_names

['Centro',
 'Capoeiras',
 'Trindade',
 'Agronômica',
 'Saco dos Limões',
 'Coqueiros',
 'Monte Cristo',
 'Jardim Atlântico',
 'Itacorubi',
 'Costeira do Pirajubaé',
 'Capivari',
 'Tapera da Base',
 'Estreito',
 'Monte Verde',
 'Balneário',
 'São João do Rio Vermelho',
 'Canto',
 'Abraão',
 'Santa Mônica',
 'Lagoa',
 'Saco Grande',
 'Córrego Grande',
 'Canasvieiras',
 'Pantanal',
 'Coloninha',
 'Barra da Lagoa',
 'Carianos',
 'José Mendes',
 'Ingleses Centro',
 'João Paulo',
 'Campeche Leste',
 'Campeche Sul',
 'Rio Tavares Central',
 'Santinho',
 'Ponta das Canas',
 'Vargem do Bom Jesus',
 'Armação',
 'Cachoeira do Bom Jesus Leste',
 'Pântano do Sul',
 'Itaguaçu',
 'Jurere Leste',
 'Campeche Norte',
 'Vargem Grande',
 'Campeche Central',
 'Ressacada',
 'Morro das Pedras',
 'Alto Ribeirão Leste',
 'Alto Ribeirão',
 'Ribeirão da Ilha',
 'Santo Antônio',
 'Sambaqui',
 'Ingleses Sul',
 'Bom Abrigo',
 'Jurere Oeste',
 'Porto da Lagoa',
 'Cachoeira do Bom Jesus',
 'Rio Tavares do Norte',
 'P

In [254]:
#preparing neighborhoods dataframe
column_names = ['Name', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Name,Latitude,Longitude


In [255]:
print("Checking for neighborhoods latitude/longitude on foursquare API...")

for name in neighborhood_names:
  geolocator = Nominatim(user_agent="foursquare_explorer")
  location = geolocator.geocode(name+","+city_name)
  #print(location)
  if location is not None:
    latitude = location.latitude
    longitude = location.longitude
    #print('The geograpical coordinate of {} are {}, {}.'.format(name, latitude, longitude))
    neighborhoods = neighborhoods.append({'Name': name,
                                            'Latitude':latitude,
                                            'Longitude':longitude}, ignore_index=True)
  
print("Done!")

Checking for neighborhoods latitude/longitude on foursquare API...
Done!


Now we have the coordinates of the neighborhoods.

In [256]:
neighborhoods.head()

,Name,Latitude,Longitude
0,Centro,-27.592925,-48.550060
1,Capoeiras,-27.597333,-48.590008
2,Trindade,-27.589383,-48.522400
3,Agronômica,-27.578145,-48.535717
4,Saco dos Limões,-27.608268,-48.534343


In [257]:
# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(city_map)  
    
city_map

In [258]:
#get list of beaches from wikipedia
wiki = requests.get('https://pt.wikipedia.org/wiki/Lista_de_praias_de_Florianópolis')
soup = bs(wiki.text, 'lxml') 
table = soup.findAll("li")

#filtering
beach_names=[]
for item in table:
      a = item.find("a")
      if a is not None:
        title = a.get('title')
        if title is not None and title.find("Prai") >= 0:
          if title.endswith(' (página não existe)'):
              newTitle = title[:-20]
              beach_names.append(newTitle)
          else:
              beach_names.append(title)

#removing the last unecessary item which is still on the list
del beach_names[-1]
beach_names

['Praia de Caieira da Barra do Sul',
 'Praia do Ribeirão da Ilha',
 'Praia da Tapera',
 'Praia de Sambaqui',
 'Praia de Santo Antônio de Lisboa',
 'Praia do Cacupé',
 'Praia do Santinho',
 'Praia dos Ingleses',
 'Praia da Lagoinha',
 'Praia Brava (Florianópolis)',
 'Praia da Ponta das Canas',
 'Praia Cachoeira do Bom Jesus',
 'Praia de Canasvieiras',
 'Praia de Canajurê',
 'Praia de Jurerê',
 'Praia do Forte (Florianópolis)',
 'Praia da Daniela',
 'Praia do Pontal',
 'Praia da Joaquina',
 'Praia do Gravatá',
 'Praia Mole',
 'Praia da Galheta',
 'Prainha (Barra da Lagoa)',
 'Praia da Barra da Lagoa',
 'Praia da Lagoa da Conceição',
 'Praia de Moçambique',
 'Praia do Campeche',
 'Praia da Ilha do Campeche',
 'Praia do Morro das Pedras',
 'Praia do Caldeirão',
 'Praia da Armação do Pântano do Sul',
 'Praia do Matadeiro',
 'Praia da Lagoinha do Leste',
 'Praia do Pântano do Sul',
 'Praia dos Açores',
 'Praia da Solidão',
 'Praia do Saquinho',
 'Praia de Naufragados',
 'Praia de Bom Abrigo'

In [259]:
#beach dataframe preparation
beaches = pd.DataFrame(columns=['Beach', 'Latitude', 'Longitude'])
beaches

,Beach,Latitude,Longitude


In [260]:
print("Checking for beaches latitude/longitude on foursquare API...")

for name in beach_names:
  geolocator = Nominatim(user_agent="foursquare_explorer")
  location = geolocator.geocode(name+", "+city_name)
  #print(location)
  if location is not None:
    latitude = location.latitude
    longitude = location.longitude
#     print('The geograpical coordinate of {} are {}, {}.'.format(name, latitude, longitude))
    beaches = beaches.append({'Beach': name,
                                            'Latitude':latitude,
                                            'Longitude':longitude}, ignore_index=True)
  
print("Done!")

Checking for beaches latitude/longitude on foursquare API...
Done!


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [261]:
# add markers to map
for lat, lng, label in zip(beaches['Latitude'], beaches['Longitude'], beaches['Beach']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(city_map)  
    
city_map

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Which neighborhood has more beaches?

In [263]:
from scipy.spatial.distance import cdist

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

df1 = neighborhoods
df2 = beaches

df1['point'] = [(x, y) for x,y in zip(df1['Latitude'], df1['Longitude'])]
df2['point'] = [(x, y) for x,y in zip(df2['Latitude'], df2['Longitude'])]

df2['closest'] = [closest_point(x, list(df1['point'])) for x in df2['point']]
df2['Neighborhood'] = [match_value(df1, 'point', x, 'Name') for x in df2['closest']]

df2 = df2.drop(columns=['point', 'closest'])
df1 = df1.drop(columns=['point'])
neighborhoods = neighborhoods.drop(columns=['point'])
df2.head()

,Beach,Latitude,Longitude,Neighborhood
0,Praia da Tapera,-27.688573,-48.568508,Tapera
1,Praia do Santinho,-27.458246,-48.374768,Santinho
2,Praia dos Ingleses,-27.429447,-48.396534,Ingleses Centro
3,Praia da Lagoinha,-27.388978,-48.426370,Lagoinha do Norte
4,Praia Brava (Florianópolis),-27.397613,-48.415825,Praia Brava


In [264]:
#now we get the counts for each neighborhood
counts = df2.groupby(['Neighborhood']).size()
counts

Neighborhood
Armação                     2
Barra da Lagoa              3
Caiacanga                   1
Campeche Leste              1
Canasvieiras                1
Canto dos Araçás            2
Coqueiros                   2
Daniela                     3
Ingleses Centro             1
Itaguaçu                    1
Jurere Leste                2
Lagoinha do Norte           2
Morro das Pedras            1
Porto da Lagoa              1
Praia Brava                 2
Pântano do Sul              4
Santinho                    1
São João do Rio Vermelho    2
Tapera                      1
dtype: int64

In [0]:
#adding the BeachesCount column to dataframe with default 0
neighborhoods.insert(3, 'BeachesCount', 0)

In [0]:
neighborhoods.set_index('Name', inplace=True) #Beach is the index

In [0]:
# Assigning the value of the count list to each neighborhood
for n in counts.index:
  neighborhoods.loc[n,'BeachesCount'] = counts[n]

In [268]:
neighborhoods.sort_values(by='BeachesCount', ascending=False).head()

,Latitude,Longitude,BeachesCount
Name,,,
Pântano do Sul,-27.779951,-48.507319,4
Daniela,-27.448954,-48.531250,3
Barra da Lagoa,-27.574778,-48.425835,3
São João do Rio Vermelho,-27.483806,-48.409638,2
Canto dos Araçás,-27.594075,-48.460940,2


#### Define Foursquare Credentials and Version

In [0]:
CLIENT_ID = 'XCZZBWCOGQ5BJ5KD3NOGA4J0LW52XU25S4IJCS5Z5GRJOGCV' # your Foursquare ID
CLIENT_SECRET = '5JWA0FLEQE0OEDHAJRXKDD2HPWBSUH5LKUASZXDPP13Q0OQ5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [0]:
manhattan_data.loc[0, 'Neighborhood']

Get the neighborhood's latitude and longitude values.

In [0]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [0]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [0]:
results = requests.get(url).json()
results

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [0]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

And how many venues were returned by Foursquare?

In [0]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [0]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [0]:
print(manhattan_venues.shape)
manhattan_venues.head()

Let's check how many venues were returned for each neighborhood

In [0]:
manhattan_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [0]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [0]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

And let's examine the new dataframe size.

In [0]:
manhattan_onehot.shape

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [0]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

#### Let's confirm the new size

In [0]:
manhattan_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues

In [0]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [0]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
# # run k-means clustering
# kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# # check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [0]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [0]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 2

In [0]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 3

In [0]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 4

In [0]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 5

In [0]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).